# Representation Learning using Auto Encoders

In [ ]:
import keras as K
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

from keras.datasets import mnist
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

In [ ]:
print("Keras version:", K.__version__)
print("TensorFlow version:", tf.__version__)

In [ ]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
if tf.config.experimental.list_physical_devices('GPU'):
    print("GPU is available")
else:
    print("GPU is not available")

In [ ]:
(train_images, _), (test_images, _) = mnist.load_data()

# reshape images to 1D
train_images = train_images.astype('float32') / 255.
test_images = test_images.astype('float32') / 255.
train_images = train_images.reshape((len(train_images), 784))
test_images = test_images.reshape((len(test_images), 784))

# add noise to images
noise_factor = 0.5
x_train_noisy = train_images + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=train_images.shape) 
x_test_noisy = test_images + noise_factor * np.random.normal(loc=0.0, scale=1.0, size=test_images.shape)

x_train_noisy = np.clip(x_train_noisy, 0., 1.)
x_test_noisy = np.clip(x_test_noisy, 0., 1.)

train_labels = K.utils.to_categorical(train_labels, 10)
test_labels = K.utils.to_categorical(test_labels, 10)

In [ ]:
# setting up the model
input_img = K.layers.Input(shape=(784,))
# encoder
encoded = K.layers.Dense(128, activation='relu')(input_img)
encoded = K.layers.Dense(64, activation='relu')(encoded)
# decoder
decoded = K.layers.Dense(128, activation='relu')(encoded)
decoded = K.layers.Dense(784, activation='sigmoid')(decoded)
autoencoder = K.Model(input_img, decoded)
encoder = K.Model(input_img, encoded)

autoencoder.compile(optimizer='adam', loss='mean_squared_error')

# early stopping
early_stopping = K.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0001,
    patience=5,
    verbose=1, 
    restore_best_weights=True)

# autoencoder's training
autoencoder.fit(x_train_noisy, train_images,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_split=0.2,
                callbacks=[early_stopping])


In [ ]:
# extract hidden representations using the encoder
encoded_train_images = encoder.predict(train_images)
encoded_test_images = encoder.predict(test_images)

# set up classifier model
classifier_input = layers.Input(shape=(64,))
classifier_output = layers.Dense(10, activation='softmax')(classifier_input)
classifier = K.Model(classifier_input, classifier_output)
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# training the classifier
classifier.fit(encoded_train_images, train_labels,
               epochs=50, 
               batch_size=256,
               validation_split=0.2,
               callbacks=[early_stopping])

# classifier's evaluation
test_loss, test_accuracy = classifier.evaluate(encoded_test_images, test_labels)
print(f'Test accuracy: {test_accuracy}, Test loss: {test_loss}')

In [ ]:
# compute the reconstruction error on the test set
mse = K.losses.MeanSquaredError()
reconstruction_error = mse(autoencoder.predict(x_test_noisy), test_images).numpy()
print(f'Reconstruction error: {reconstruction_error}')

In [ ]:
# generate a noise image and iteratively denoise it
noise_image = np.random.normal(loc=0, scale=1, size=(1, 784))
noise_image = np.clip(noise_image, 0., 1.)

for i in range(100):  # Number of iterations can be adjusted
    noise_image = autoencoder.predict(noise_image)
    if i % 10 == 0:
        print(f"Iteration {i}")

# plot the denoised image
plt.imshow(noise_image.reshape(28, 28), cmap='gray')
plt.title("Denoised Image")
plt.show()
